In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve import Draw
from netgen.gui import *

import math

# Parameters
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilheight = nwindings * (2 * wireradius) * 1.2  # Length of the coil
coilradius = 0.01  # Radius of the coil
airh = 10 * coilradius  # Dimension of the surrounding air

# Create the cylinder face for the helix path
cyl = Cylinder((0, 0, -coilheight / 2), Z, r=coilradius, h=coilheight).faces[0]

# Create the helical edge
heli = Edge(Segment((0, 0), (nwindings * 2 * math.pi, coilheight)), cyl)

# Get start and end points and tangents (optional, can be used for further modifications)
ps = heli.start
vs = heli.start_tangent
pe = heli.end
ve = heli.end_tangent

# Create the helical wire and circular face for the coil cross-section
spiral = Wire([heli])
circ = Face(Wire([Circle(ps, Y, wireradius)]))

# Generate the coil by sweeping the circular face along the helical wire
coil = Pipe(spiral, circ)

# Set mesh size and names for the coil faces
coil.faces.maxh = 0.2
coil.faces.name = "coilbnd"
coil.faces.Max(Z).name = "in"
coil.faces.Min(Z).name = "out"
coil.mat("coil")

# Calculate cross-sectional area (optional)
crosssection = coil.faces.Max(Z).mass

# Create the surrounding air box and subtract the coil
box = Box((-airh / 2, -airh / 2, -airh / 2), (airh / 2, airh / 2, airh / 2))
box.faces.name = "outer"
air = box - coil
air.mat("air")

# Combine the coil and air into a single geometry
geo = OCCGeometry(Glue([coil, air]))

# Display the geometry
Draw(geo)

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2406
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 16 thread(s)


 Face 1 / 9 (parameter space projection)
 Face 2 / 9 (parameter space projection)
 Face 3 / 9 (parameter space projection)
 Face 4 / 9 (parameter space projection)
 Face 5 / 9 (parameter space projection)
 Face 6 / 9 (parameter space projection)
 Face 7 / 9 (parameter space projection)
 Face 8 / 9 (parameter space projection)
 Face 9 / 9 (parameter space projection)
 Delaunay meshing
 Remove Illegal Elements
 Delaunay meshing
 start tetmeshing
 Success !
 Volume Optimization
 Meshing done, time = 20.3308 sec
ERROR: invalid command name "NGS_SolvePDE"
errinfo: invalid command name "NGS_SolvePDE"
    while executing
"NGS_SolvePDE"
    invoked from within
".ngmenu.solve invoke "Solve PDE""
    invoked from within
".bubar.solve invoke "
    invoked from within
".bubar.solve instate !disabled { .bubar.solve invoke } "
    invoked from within
".bubar.solve instate pressed { .bubar.solve state !pressed; .bubar.solve instate !disabled { .bubar.solve invoke } } "
    (command bound to event)
Th